# DATA process

In [1]:
import sys
import numpy as np
import pandas as pd
import networkx as nx
graph_1_list = []
graph_2_list = []
graph_3_list = []
graph_4_list = []
graph_5_list = []
graph_6_list = []
graph_1_list_node_1 = []
graph_2_list_node_1 = []
graph_3_list_node_1 = []

with open('graph_1.txt','r') as f:
    for line in f:
        graph_1_list.append(list(line.strip('\n').split(',')))

with open('graph_1.txt','r') as f:
    for line in f:
        graph_1_list_node_1.append(list(line.strip('\n').split(',')))
    append_list_1 = [['2', '1'], ['1', '3'], ['1', '4'], ['1', '5'], ['1', '6'], ['3', '1'], ['4', '1'], ['5', '1'], ['6', '1']]
    for edge in append_list_1:
        graph_1_list_node_1.append(edge)
        
with open('graph_2.txt','r') as f:
    for line in f:
        graph_2_list.append(list(line.strip('\n').split(',')))
        
with open('graph_2.txt','r') as f:
    for line in f:
        graph_2_list_node_1.append(list(line.strip('\n').split(',')))
    append_list_2 = [['2', '1'], ['1', '3'], ['1', '4'], ['1', '5'], ['3', '1'], ['4', '1'],]
    for edge in append_list_2:
        graph_2_list_node_1.append(edge)

with open('graph_3.txt','r') as f:
    for line in f:
        graph_3_list.append(list(line.strip('\n').split(',')))
with open('graph_3.txt','r') as f:
    for line in f:
        graph_3_list_node_1.append(list(line.strip('\n').split(',')))
    append_list_3 = [['1', '3'], ['1', '4'], ['3', '1'], ['4', '1']]
    for edge in append_list_3:
        graph_3_list_node_1.append(edge)

with open('graph_4.txt','r') as f:
    for line in f:
        graph_4_list.append(list(line.strip('\n').split(',')))
with open('graph_5.txt','r') as f:
    for line in f:
        graph_5_list.append(list(line.strip('\n').split(',')))
with open('graph_6.txt','r') as f:
    for line in f:
        graph_6_list.append(list(line.strip('\n').split(',')))


data_txt = np.loadtxt('IBM_testing_dataset.txt')
IBM_testing_txtDF = pd.DataFrame(data_txt)
IBM_data = []

for IBM_item,customer in enumerate(IBM_testing_txtDF[0]):
    if len(IBM_data) < int(customer):
        IBM_data.append(list())
        IBM_data[int(customer)-1].append(str(IBM_testing_txtDF[2][IBM_item]))
    else:
        IBM_data[int(customer)-1].append(str(IBM_testing_txtDF[2][IBM_item]))

In [2]:
import csv
import pandas as pd
from itertools import combinations
import time

IBM_data =pd.DataFrame(index=range(len(IBM_data)),data=IBM_data)
#Read data from CSV
# data = pd.read_csv('groceries.csv')
data = IBM_data
# data = pd.read_csv('Market_Basket_Optimisation.csv', header=None)
#
#Parameters
minsup = 0.045#float(input("Support-Threshold: "))
minsup = minsup * len(data)
minconf = 0.1#float(input("Confidence-Threshold: "))
# print(len(data))#9835
time_start = time.time()

#Add all data in a list of lists
items = []  
# print(len(data.values[0]))#1列(ROW)有32的物品
for i in range(0, len(data)):  #將所有data中的物品一列一列放到items
     items.append([str(data.values[i, j]) for j in range(0, len(data.values[0]))])

#Creating a list of dictionaries
count = [dict() for x in range(len(data.values[0]) + 1)]
# print(items)
# #Count support for each individual items
s = []
for i in items:
    for j in i:      #讀取所有items
        s.append(j)
for i in s:                      #創字典 放在count[1]
    #If item is present in dictionary, increment its count by 1
    if i in count[1]:
        count[1][i] = count[1][i] + 1
    #If item is not present in dictionary, set its count to 1
    else:
        count[1][i] = 1     #{'i':1}

# #Remove infrequent and empty items #去除小於minsup,nan
for i in count[1].copy():
    if(count[1][i] < minsup):
        count[1].pop(i)
# count[1].pop('nan') #去除以key = 'NAN'產生的值
count[1].pop('None') #去除以key = 'NAN'產生的值

# #Storing transactions as lists without infrequent items
a = list(count[1]) #列出所有符合的KEY
item = [list() for i in range(len(data))] #創造len(data) = 9835個list 
c = 0 
for i in range(0,len(items)): ##將所有data中的物品一列一列放到items
    for j in range(len(items[i])):
#         print(items[i][j])
        if(a.__contains__(items[i][j]) != 0): #確認物品是否存在
            item[i].append(items[i][j]) #將物品分成[['a', 'b', 'c'],[]]與itmes不同的地方是裡練已經沒有NAN了

#Function to sort list to support
def sort(a): #a = 所有符合的KEY的list
    for i in range(len(a) - 1):
        for j in range(len(a) - i - 1):
            if(count[1][a[j]] < count[1][a[j + 1]]): #如果下一個比上一個大則互換位子
                a[j],a[j + 1] = a[j + 1],a[j]
                
#Call function to sort all transactions in descending order of their support
for i in range(0,len(data)): #9835
    if(len(item[i]) > 1): #如果不只一個物件就排序他
        sort(item[i])

#Tree class for FP-Tree
class tree: #設計樹根，分支
    def __init__(self, name, sup, parent):
        self.name = name
        self.sup = sup
        self.nodeLink = None
        self.parent = parent
        self.children = []

#Function to check if the node is present is a child of the current node
def ispresent(node,name):    #確認是否有children
    f = -1               
    for i in node.children:
        f += 1
        if(i.name == name):
            return f
    return -1

#HeaderTable which stores the reference of last/first occurence of an item. Used as a linked list to generate candidate trees 
lastocc = count[1].copy() #符合sup的字典
for i in lastocc: #lastocc字典中所有值皆為NONE
    lastocc[i] = None
#Function to create FP-tree
root = tree("root", -1, None)
z = 0
for i in item: #item為沒有NAN的items
    current = root
    for j in range(len(i)): #len(i)9835
        if(ispresent(current,i[j]) >= 0): #第一次通常不符合此情形，直接執行else，看有沒有children
            current = current.children[ispresent(current, i[j])]
            current.sup = current.sup + 1
        else:
            child=tree(i[j], 1, current)    #創建i[j]子根
            current.children.append(child) #將子根放入children =[] 
            t = current #t 記錄柱上一層的tree1
            current = current.children[ispresent(current, i[j])] #現在current.children裡有子根，所以self.name = i[j]如下示意，
            #ispresent(current,i[j])回傳 0 ，現在的current為tree2
            current.parent = t #紀錄上一層tree1
            if(lastocc[current.name] == None):
                lastocc[current.name] = current #創建{'i[j]': tree2-1()} 
            else:#創出第一層並列樹枝
                current.nodeLink = lastocc[current.name] #將tree2-2連結到tree4
                lastocc[current.name] = current #CURRENT = TREE2-2 {'i[j]': tree2-2}
            
# class tree1:
#     def __init__(self, name, sup, parent):
#         self.name = 'root'
#         self.sup = -1
#         self.nodeLink = None
#         self.parent = none
#         self.children = [
#   class tree2-1:
#     def __init__(self, name, sup, parent):      [['A','B','C'],['C','A']]
#         self.name = 'A'
#         self.sup = 1
#         self.nodeLink = NONE
#         self.parent = parent
#         self.children = [
#                                     class tree3:
#                                         def __init__(self, name, sup, parent):
#                                             self.name = 'B'
#                                             self.sup = 1
#                                             self.nodeLink = None
#                                             self.parent = parent
#                                             self.children = [
#                                                                 class tree4:
#                                                                     def __init__(self, name, sup, parent):
#                                                                         self.name = 'B'
#                                                                         self.sup = 1
#                                                                         self.nodeLink = None
#                                                                         self.parent = parent
#                                                                         self.children = []
#                                                                 ]
#         ],
#   class tree2-2:
#     def __init__(self, name, sup, parent):
#         self.name = 'C'
#         self.sup = sup
#         self.nodeLink = TREE 4
#         self.parent = TREE1
#         self.children = []
# ]



#Function to get frequent itemsets with suffix 'node' and length n
def singlepath(node, n):
    c = 0
    sup = node.sup
    path = []
    pathname = []
    current = node
    
    #Get the path from current node to root
    while(current.parent != None): #查看目current是否為'root'，如果是則無parent，重複做直道回到最上層('root')
        path.append(current)     #將current內容紀錄(tree)
        pathname.append(current.name)      #('c')
        current = current.parent  #往上一層移動
    path.remove(node) #除了自己以外的路線
    pathname.remove(node.name) 
    candidatepath = []
    temp_candidatepath = []
   
    #Generate combinations of length n in the path
    a = (list(combinations(pathname, n))) #列出除了自己以外的長度為n的組合
    for j in a:
        temp_candidatepath.append(tuple(sorted(j)))   #以tuple形式存在temp_candidatapath裡
    #Append the suffix 'node.name' to the above paths
    for j in temp_candidatepath:
        j = list(j) #再次變回list
        j.append(node.name) #將原本剔除的當前TREE名加回去 變成包含當前的組合，就是在做freqent pattern
        candidatepath.append(sorted(j)) #在加回去變成 [['a','c'],['b','c']]
    #Update counts of the generated itemsets
    for j in candidatepath:
        j = tuple(j) #j = ('a','c')
        if j in count[n + 1]: #count[1]為符合minsup的字典，n至少會從1開始 
            count[n + 1][j] = count[n + 1][j] + sup  # 將之前已創好的value值，加上當前tree所有的sup
        else:
            count[n + 1][j] = sup #創建新字典,此sup為當前tree存有的sup
            
    #Iterating in the candidate tree recursively 
    if(node.nodeLink != None): #查看是否存在分支
        node=node.nodeLink
        singlepath(node, i) #i為組合長度
    
#Check if itemset is frequent
def frequent(n): #算出現次數
    f=0
    for i in count[n]:
        if(count[n][i] >= minsup):
            f = 1
    if(f == 1):
        return 1
    else:
        return 0

#Call singlepath function for all frequent nodes
for i in range(1, len(data.values[0]) + 1): #1列(ROW)有32的物品, +1是確保有32為一組的組合  #!
    if(frequent(i) == 1):
        for j in lastocc: #開始製造組合，將結果放到count[]裡，count唯有32空字典的list
            singlepath(lastocc[j], i)

#Remove infrequent itemsets
for z in range(len(data.values[0]) + 1):            
    for i in count[z].copy():
            if(count[z][i] < minsup): #查看所有的dict裡面的東西，是否符合minsup
                count[z].pop(i) #去掉不符合的

frequent_pattern = []
for items in count:
    for want in items:
        frequent_pattern.append(want)
                
#Get 'q', the length of the longest itemset
i=1 #設2是因為從第二筆的字典，才開始有東西
while(len(count[i]) != 0): #從這邊就能知道最大符合長度是多長，因為不符合的都被刪掉了，所以count最大的子矩陣
    i = i + 1
q = i - 1

#Find maximal and closed itemsets
maximal = []
closed = []
for i in range(1, q):
    for j in count[i]:
        fm = 0
        fc = 0
        for k in count[i + 1]:
            a = set(list([j]))
            b = set(list(k))
            #Set is maximal if no immediate superset is frequent
            if(a.intersection(b) == a): #如果兩者重疊則maximal
                fm = 1 
                #Set is closed if none of its immediate supersets have equal support
                if(count[i][j] == count[i + 1][k]): #出現次數相同
                    fc = 1
        if(fm == 0):
            maximal.append(j)
        if(fc == 0):
            closed.append(j)
#All sets at the top of the tree are automatically maximal and closed
for i in count[q]:
    maximal.append(i)
    closed.append(i)
    
print("frequent_pattern")
print('number of frequent pattern',len(frequent_pattern))
# print(frequent_pattern)
 
#Find Association Rules 
print("Association_Rules")
all_frequent_pattern = {}
for len_FP in count:
    all_frequent_pattern.update(len_FP)

IBM_list = []
def rulegenerator(fitems): #fitmes = 最終dic
    '''
    Generates association rules from the frequent itemsets
    '''
    counter = 0
    
    for itemset in fitems.keys():
        if isinstance(itemset, str): #只有單個得我不要，我要tuple形式('a','b')
            continue
        length = len(itemset)#tuple裡面有幾項
        union_support = fitems[tuple(itemset)]#原本itemset為LIST形式，必須使用tuple(才可以變成key ，看兩個以上的品項的出現次數
        for i in range(1, length):
            lefts = map(list, combinations(itemset, i)) #['mineral water'], ['pancakes']
            for left in lefts:
                if len(left) == 1:
                    if ''.join(left) in fitems:
                        leftcount = fitems[''.join(left)]#單品項出現次數 
                        conf = union_support / leftcount
                y_list = []
                x_list = []
                
                if conf >= minconf:
                    right = list(itemset[:])
                    sub_list = []
                    for e in left:                          #去除被分配在左邊的品項
                        right.remove(e)     
#                     fo.write(str(left) + ' (' + str(leftcount) + ')' + ' -> ' + str(right) + ' (' + str(fitems[''.join(right)]) + ')' + ' [' + str(conf) + ']' + '\n')                    
#                     print(str(left) + ' -> ' + str(right) + ' (' + str(conf) + ')')
                    
#                     for y in left:
#                         y_list.append(int(float(y)))
#                     for x in right:
#                         x_list.append(int(float(x)))
                    for y in left:
                        y_list.append(y)
                    for x in right:
                        x_list.append(x)   
    
                    
                    sub_list.append(y_list)
                    sub_list.append(x_list)
                    IBM_list.append(sub_list)
                    counter += 1
#                     fo.close()
    print(counter, "rules generated")

rulegenerator(all_frequent_pattern)





time_end = time.time()
print('FP_Growth_spent_time', time_end - time_start)
# print( time_end - time_start)
# print(len(ant))

frequent_pattern
number of frequent pattern 171
Association_Rules
1088 rules generated
FP_Growth_spent_time 0.30556511878967285


# graph_1

In [3]:
import numpy as np
import time

start_time = time.time()

want_list = graph_1_list
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()

times = 0

while True:
    times += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_1_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_1_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)
    

2
0.0
6
6
[0.2 0.2 0.2 0.2 0.2 0. ]
[0.  0.2 0.2 0.2 0.2 0.2]
time cost 0.0017290115356445312


# graph_1_power_node_1

In [4]:
import numpy as np
import time

start_time = time.time()

want_list = graph_1_list_node_1
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()

times = 0

while True:
    times += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_1_HITS_power_node1_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_1_HITS_power_node1_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)
    

27
8.18748695518845e-05
6
6
[0.26960288 0.15772303 0.15772303 0.15772303 0.15772303 0.09950499]
[0.26960288 0.09950499 0.15772303 0.15772303 0.15772303 0.15772303]
time cost 0.003366708755493164


# graph_2

In [5]:
import numpy as np
import time

start_time = time.time()

want_list = graph_2_list
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_2_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_2_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)

2
0.0
5
5
[0.2 0.2 0.2 0.2 0.2]
[0.2 0.2 0.2 0.2 0.2]
time cost 0.0015187263488769531


# graph_2_power_node_1

In [6]:
import numpy as np
import time

start_time = time.time()

want_list = graph_2_list_node_1
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()

times = 0

while True:
    times += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_2_HITS_power_node1_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_2_HITS_power_node1_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)
    

25
9.679258444884875e-05
5
5
[0.28898506 0.19785787 0.19785787 0.19785787 0.11744132]
[0.28898506 0.11744132 0.19785787 0.19785787 0.19785787]
time cost 0.006096839904785156


# graph_3

In [7]:
import numpy as np
import time

start_time = time.time()
want_list = graph_3_list
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_3 = 0    
while True:    
    times_graph_3 += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times_graph_3}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_3_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_3_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)

11
4.553682221275501e-05
4
4
[0.19098143 0.30901857 0.30901857 0.19098143]
[0.19098143 0.30901857 0.30901857 0.19098143]
time cost 0.0052988529205322266


# graph_3_power_node_1

In [8]:
import numpy as np
import time

start_time = time.time()

want_list = graph_3_list_node_1
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()

times = 0

while True:
    times += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_3_HITS_power_node1_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_3_HITS_power_node1_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)
    

18
8.922142006284606e-05
4
4
[0.28077218 0.21922782 0.28077218 0.21922782]
[0.28077218 0.21922782 0.28077218 0.21922782]
time cost 0.0052492618560791016


# graph_4

In [9]:
import numpy as np
import time

start_time = time.time()
want_list = graph_4_list
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_4 = 0    
while True:    
    times_graph_4 += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times_graph_4}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_4_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_4_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)

18
5.615704904932822e-05
7
7
[0.27544014 0.04777115 0.10868865 0.19864916 0.18374474 0.11673865
 0.06896751]
[0.13949234 0.17790865 0.20082167 0.14017854 0.20142026 0.05609268
 0.08408586]
time cost 0.006799221038818359


# graph_5

In [10]:
import numpy as np
import time

start_time = time.time()
want_list = graph_5_list
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_5_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_5_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)

18
5.789337689870861e-05
469
469
[6.46036586e-11 1.64613565e-17 2.90103542e-12 2.38033976e-07
 6.41389290e-07 3.11833712e-12 1.08354930e-06 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.55663523e-11 1.02130581e-06
 3.21510869e-20 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.02632035e-11 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.45391334e-11 0.00000000e+00
 0.00000000e+00 1.48067541e-13 0.00000000e+00 0.00000000e+00
 2.48747675e-02 2.00966450e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 9.56044344e-08 0.00000000e+00
 0.00000000e+00 0.00000000e+00 6.08503969e-13 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.65793291e-02
 0.00000000e+00 1.92848847e-02 0.00000000e+00 9.01822015e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.0000

# graph_6

In [11]:
import numpy as np
import time

start_time = time.time()
want_list = graph_6_list
want_threshold_error = 0.0001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_6_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_6_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)

96
9.739443523008859e-05
1228
1228
[0.00269366 0.         0.         ... 0.         0.00973479 0.        ]
[0.00000000e+00 9.02424937e-04 1.08378076e-31 ... 1.50576859e-04
 1.36505555e-02 2.46923956e-31]
time cost 0.6147444248199463


# IBM

In [14]:

node_list = {}
for idx, x in enumerate(all_frequent_pattern.keys()):
    node_list[x] = idx


In [18]:
import numpy as np
import time

start_time = time.time()

want_list = IBM_list
want_threshold_error = 0.0001


dim = len(node_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx,transaction in enumerate(want_list):
    if (len(transaction[0]) != 1):
        x_location = node_list[tuple(transaction[0])]
    elif len(transaction[1]) != 1:
#         print(transaction, len(transaction[0]), len(transaction[1]))
        y_location = node_list[tuple(transaction[1])]
    else:
        x_location = node_list[transaction[0][0]]
        y_location = node_list[transaction[1][0]]
    
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"IBM_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"IBM_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority

19
9.083209886661452e-05
171
171
[3.77879529e-09 8.99151493e-09 8.99151493e-09 3.77879529e-09
 3.77879529e-09 1.26269555e-12 1.26269555e-12 1.26269555e-12
 1.26269555e-12 1.26269555e-12 1.26269555e-12 1.65854767e-19
 1.94110343e-19 8.94127581e-20 3.88980268e-16 3.88980268e-16
 3.88980268e-16 4.90816452e-16 1.65854767e-19 0.00000000e+00
 2.17002209e-16 2.17002209e-16 6.62016123e-26 6.62016123e-26
 8.73936457e-09 4.88760358e-07 4.83512197e-07 4.83512197e-07
 4.83512197e-07 2.03820197e-08 0.00000000e+00 7.17113436e-05
 3.22064344e-05 3.33144631e-04 4.54379299e-04 5.22695398e-04
 3.58208438e-05 9.69330868e-05 2.60188691e-04 5.45476642e-05
 4.82164354e-05 6.76128963e-03 3.52297410e-02 7.20502628e-03
 3.06783640e-02 9.39280897e-03 3.08034196e-02 2.46558069e-02
 2.34346098e-02 9.39779750e-03 2.30800958e-02 1.46682820e-02
 1.50836349e-02 8.51980605e-03 1.52090687e-02 1.53097065e-02
 2.88582167e-05 0.00000000e+00 2.26512262e-03 2.32039656e-03
 5.74844329e-05 5.74339139e-05 2.24959071e-03 5.7201

# 備案

In [16]:
# import numpy as np
# import time

# start_time = time.time()

# want_list = np.array(IBM_list)
# want_threshold_error = 0.0001
# def find_dim(wnat_list):
#     dim = np.max(want_list)
#     return dim

# dim = int(float(find_dim(want_list)[0]))
# adj_matrix_hub = np.zeros((dim, dim))


# last_hub = last_authority =np.ones(dim)
# # 建hub_matrix
# for idx,transaction in enumerate(want_list):
#     for x_location in transaction[0]:
#         for y_location in transaction[1]:
#             y_location = y_location - 1
#             x_location = x_location - 1
#             adj_matrix_hub[y_location][x_location] += 1

# adj_matrix_authority = adj_matrix_hub.transpose()
# times_graph_2 = 0    
# while True:    
#     times_graph_2 += 1
#     new_hub = []
#     new_authority = []
# #     print(last_hub)
# #     print(last_authority)
#     for i in range(len(adj_matrix_hub)):
#         new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
#     for i in range(len(adj_matrix_authority)):
#         new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
#     new_hub_value = sum(np.absolute(new_hub))
#     last_hub_value = sum(np.absolute(last_hub))
#     new_hub = new_hub/new_hub_value
#     dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
#     new_authority_value = sum(np.absolute(new_authority))
#     last_authority_value = sum(np.absolute(last_authority))
#     new_authority = new_authority/new_authority_value
    
#     dif_authority = sum(np.absolute(new_authority - last_authority))
# #     print(new_authority)
# #     print(last_authority)
# #     print(sum(np.absolute(new_authority - last_authority)))
# #     print(new_hub)
# #     print(last_hub)
# #     print(sum(np.absolute(new_hub - last_hub)))
#     if (dif_hub + dif_authority) < want_threshold_error:
#         print(f'{times_graph_2}')
#         print(dif_hub+dif_authority)
#         print(len(new_authority))
#         print(len(new_hub))
#         print(new_hub)
#         print(new_authority)
#         end_time = time.time()
#         spend_time = end_time - start_time 
#         print(f'time cost {spend_time}')
#         np.savetxt(f"IBM_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
#         np.savetxt(f"IBM_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
#         break
    
#     last_hub = new_hub
#     last_authority = new_authority

# static_test

# graph_1

In [98]:

print(want_threshold_error)
print(times)
# print(new_hub)
# print(new_authority)
print(f'time cost {spend_time}')

1e-07
18
time cost 0.002382993698120117


In [97]:
import numpy as np
import time

start_time = time.time()

want_list = graph_3_list
want_threshold_error = 0.0000001
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()

times = 0

while True:
    times += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_1_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"graph_1_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority
#     print(new_hub,new_authority)
    

18
5.4017781914161844e-08
4
4
[0.19098301 0.30901699 0.30901699 0.19098301]
[0.19098301 0.30901699 0.30901699 0.19098301]
time cost 0.002382993698120117


In [67]:
import numpy as np
import time

start_time = time.time()

want_list = IBM_list
want_threshold_error = 0.0001


dim = len(node_list)
adj_matrix_hub = np.zeros((dim, dim))


last_hub = last_authority =np.ones(dim)
# 建hub_matrix
for idx,transaction in enumerate(want_list):
    if (len(transaction[0]) != 1):
        x_location = node_list[tuple(transaction[0])]
    elif len(transaction[1]) != 1:
#         print(transaction, len(transaction[0]), len(transaction[1]))
        y_location = node_list[tuple(transaction[1])]
    else:
        x_location = node_list[transaction[0][0]]
        y_location = node_list[transaction[1][0]]
    
    adj_matrix_hub[y_location][x_location] += 1

adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_hub = []
    new_authority = []
#     print(last_hub)
#     print(last_authority)
    for i in range(len(adj_matrix_hub)):
        new_hub.append(np.dot(adj_matrix_hub[i], last_authority))
    for i in range(len(adj_matrix_authority)):
        new_authority.append(np.dot(adj_matrix_authority[i], last_hub))
    
    new_hub_value = sum(np.absolute(new_hub))
    last_hub_value = sum(np.absolute(last_hub))
    new_hub = new_hub/new_hub_value
    dif_hub = sum(np.absolute(new_hub - last_hub)) 
    
    new_authority_value = sum(np.absolute(new_authority))
    last_authority_value = sum(np.absolute(last_authority))
    new_authority = new_authority/new_authority_value
    
    dif_authority = sum(np.absolute(new_authority - last_authority))
#     print(new_authority)
#     print(last_authority)
#     print(sum(np.absolute(new_authority - last_authority)))
#     print(new_hub)
#     print(last_hub)
#     print(sum(np.absolute(new_hub - last_hub)))
    if (dif_hub + dif_authority) < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_hub+dif_authority)
        print(len(new_authority))
        print(len(new_hub))
        print(new_hub)
        print(new_authority)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"IBM_HITS_hub.txt", new_hub, newline = " ",fmt = '%10.5f')
        np.savetxt(f"IBM_HITS_authority.txt", new_authority, newline = " ",fmt = '%10.5f')
        break
    
    last_hub = new_hub
    last_authority = new_authority

19
9.083209886661452e-05
171
171
[3.77879529e-09 8.99151493e-09 8.99151493e-09 3.77879529e-09
 3.77879529e-09 1.26269555e-12 1.26269555e-12 1.26269555e-12
 1.26269555e-12 1.26269555e-12 1.26269555e-12 1.65854767e-19
 1.94110343e-19 8.94127581e-20 3.88980268e-16 3.88980268e-16
 3.88980268e-16 4.90816452e-16 1.65854767e-19 0.00000000e+00
 2.17002209e-16 2.17002209e-16 6.62016123e-26 6.62016123e-26
 8.73936457e-09 4.88760358e-07 4.83512197e-07 4.83512197e-07
 4.83512197e-07 2.03820197e-08 0.00000000e+00 7.17113436e-05
 3.22064344e-05 3.33144631e-04 4.54379299e-04 5.22695398e-04
 3.58208438e-05 9.69330868e-05 2.60188691e-04 5.45476642e-05
 4.82164354e-05 6.76128963e-03 3.52297410e-02 7.20502628e-03
 3.06783640e-02 9.39280897e-03 3.08034196e-02 2.46558069e-02
 2.34346098e-02 9.39779750e-03 2.30800958e-02 1.46682820e-02
 1.50836349e-02 8.51980605e-03 1.52090687e-02 1.53097065e-02
 2.88582167e-05 0.00000000e+00 2.26512262e-03 2.32039656e-03
 5.74844329e-05 5.74339139e-05 2.24959071e-03 5.7201